In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
import os
import boto3
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import csv
import numpy as np
from pathlib import Path

In [3]:
def maybe_pickle(data, filename, force=False):
    if os.path.exists(filename) and not force:
        # You may override by setting force=True.
        print('%s already present - Skipping pickling.' % filename)
    else:
        print('Pickling %s.' % filename)
        try:
            with open(filename, 'wb') as f:
                pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to', filename, ':', e)


def maybe_download(table, force=False):
    storage_file = table + ".pkl"
    if force or not os.path.exists(storage_file):
        data = data_retrieve(table)
        maybe_pickle(data, storage_file, force)
    return storage_file
        
def data_retrieve(table, page_size=100):
    """download data from table"""
    client = boto3.client('dynamodb')
    paginator = client.get_paginator('scan')

    counter = 0
    items = []
    for page in paginator.paginate(TableName=table, PaginationConfig={"PageSize": page_size}):
        items.extend(page["Items"])
        
        counter += 1
        if counter % 50 == 0:  # print debug every 50 pages
            print("items: {}. next page. {}. Count {}. ScannedCount: {}".format(len(items), counter, page["Count"], page["ScannedCount"]))


    return items
    

In [4]:
# data_file = maybe_download("apthunt")
data_file = maybe_download("apthunt")
data = pickle.load(open(data_file, 'rb'))

items: 5000. next page. 50. Count 100. ScannedCount: 100
items: 10000. next page. 100. Count 100. ScannedCount: 100
items: 15000. next page. 150. Count 100. ScannedCount: 100
items: 20000. next page. 200. Count 100. ScannedCount: 100
items: 25000. next page. 250. Count 100. ScannedCount: 100
items: 30000. next page. 300. Count 100. ScannedCount: 100
items: 35000. next page. 350. Count 100. ScannedCount: 100
items: 40000. next page. 400. Count 100. ScannedCount: 100
items: 45000. next page. 450. Count 100. ScannedCount: 100
items: 50000. next page. 500. Count 100. ScannedCount: 100
items: 55000. next page. 550. Count 100. ScannedCount: 100
items: 60000. next page. 600. Count 100. ScannedCount: 100
items: 65000. next page. 650. Count 100. ScannedCount: 100
items: 70000. next page. 700. Count 100. ScannedCount: 100
items: 75000. next page. 750. Count 100. ScannedCount: 100
items: 80000. next page. 800. Count 100. ScannedCount: 100
items: 85000. next page. 850. Count 100. ScannedCount: 100

items: 680000. next page. 6800. Count 100. ScannedCount: 100
items: 685000. next page. 6850. Count 100. ScannedCount: 100
items: 690000. next page. 6900. Count 100. ScannedCount: 100
items: 695000. next page. 6950. Count 100. ScannedCount: 100
items: 700000. next page. 7000. Count 100. ScannedCount: 100
items: 705000. next page. 7050. Count 100. ScannedCount: 100
items: 710000. next page. 7100. Count 100. ScannedCount: 100
items: 715000. next page. 7150. Count 100. ScannedCount: 100
items: 720000. next page. 7200. Count 100. ScannedCount: 100
items: 725000. next page. 7250. Count 100. ScannedCount: 100
items: 730000. next page. 7300. Count 100. ScannedCount: 100
items: 735000. next page. 7350. Count 100. ScannedCount: 100
items: 740000. next page. 7400. Count 100. ScannedCount: 100
items: 745000. next page. 7450. Count 100. ScannedCount: 100
items: 750000. next page. 7500. Count 100. ScannedCount: 100
items: 755000. next page. 7550. Count 100. ScannedCount: 100
items: 760000. next page

In [5]:
print("wow")

wow


In [6]:
# To go from dynamo format to python
from boto3.dynamodb import types

deserializer = types.TypeDeserializer()
python_data = [{k: deserializer.deserialize(v) for k,v in d.items()} for d in data]

In [7]:
del data

In [8]:
df = pd.DataFrame(python_data)

In [9]:
del python_data

In [ ]:
with open("df.pkl", 'wb') as f:
    pickle.dump(df, f, pickle.HIGHEST_PROTOCOL)


## can restore from here

In [3]:
df = pickle.load(open("df.pkl", 'rb'))

In [4]:
df.head()

,parsed_postingtitletext,parsed_map_address,added,PostUrl,PostTitle,parsed_price,parsed_price_text,parsed_housing,PostContent,parsed_data_longitude,...,parsed_bedrooms,parsed_furnished,parsed_wd,parsed_type,parsed_laundryb,parsed_nthumbs,parsed_garagea,parsed_laundrys,parsed_garaged,parsed_catsok
0,$3150 / 2br - 1300ft2 - Sunset Oaks Townhome- ...,1249 Crescent Terrace,1538706586266,https://sfbay.craigslist.org/sby/apa/d/sunset-...,simulated,3150,$3150,2br - 1300ft2,simulated,-122.026428,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,$1769 / 2br - 875ft2 - ❄ Pittsburg Apartment L...,2205 E Leland Rd,1539451812116,https://sfbay.craigslist.org/eby/apa/d/pittsbu...,simulated,1769,$1769,2br - 875ft2,simulated,-121.864673,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,$3000 / 2br - 1800ft2 - 2 Bedroom 1 Bath 1 Car...,375 Pope Street,1538947757137,https://sfbay.craigslist.org/sfc/apa/d/2-bedro...,simulated,3000,$3000,2br - 1800ft2,simulated,-122.439706,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,$2150 / 3br - 1200ft2 - Updated 1200 SQ FT 3 B...,2436 Baltic Dr,1585287116209,https://sfbay.craigslist.org/eby/apa/d/fairfie...,simulated,2150,$2150,3br - 1200ft2,simulated,-122.019156,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,$2150 / 1br - 726ft2 - FULLY RENOVATED 1BED 1B...,1182 E Street near 2nd,1587684611249,https://sfbay.craigslist.org/eby/apa/d/hayward...,simulated,2150,$2150,1br - 726ft2,simulated,-122.076737,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# small cleanup of "simulated"
del df["FeedTitle"]
del df["FeedUrl"]
del df["PostContent"]
del df["PostPublished"]
del df["PostTitle"]
# delete parsed_notices. looks like it is the same
del df["parsed_notices"]
# no need for price text
del df["parsed_price_text"]

In [6]:
df.columns

Index(['parsed_postingtitletext', 'parsed_map_address', 'added', 'PostUrl',
       'parsed_price', 'parsed_housing', 'parsed_data_longitude',
       'parsed_attrs', 'parsed_district', 'parsed_map_link',
       'parsed_postingbody', 'parsed_page_head', 'parsed_thumbs',
       'parsed_titletextonly', 'intid', 'parsed_data_latitude',
       'parsed_dogsok', 'parsed_area', 'parsed_bedrooms', 'parsed_furnished',
       'parsed_wd', 'parsed_type', 'parsed_laundryb', 'parsed_nthumbs',
       'parsed_garagea', 'parsed_laundrys', 'parsed_garaged', 'parsed_catsok'],
      dtype='object')

In [7]:
def clean_parsed_args(args):
    if not isinstance(args, list):
        return []
    clean = []
    for p in args:
        # remove days
        if p.startswith("friday") or p.startswith("monday") or p.startswith("saturday") or p.startswith("sunday") or p.startswith("thursday") or p.startswith("tuesday") or p.startswith("wednesday"):
            continue
        # remove br/ba
        if "BR / " in p: continue
        # remove ft2
        if p.endswith("ft2"): continue

        # remove application fee
        if p.startswith("application fee"): continue
        
        # remove broker fee
        if p.startswith("broker fee"): continue
        
        # remove available
        if p.startswith("available"): continue
        
        # remove listed by
        if p.startswith("listed by"): continue
        clean.append(p)
    return clean
        

In [8]:
df['parsed_attrs'] = df['parsed_attrs'].map(clean_parsed_args)
# one_hot = pd.get_dummies(df['parsed_attrs'])

In [9]:
df['parsed_attrs']

0          [townhouse, w/d in unit, no smoking, detached ...
1                                                [apartment]
2          [house, laundry in bldg, no smoking, attached ...
3                      [house, w/d hookups, detached garage]
4          [cats are OK - purrr, dogs are OK - wooof, apa...
                                 ...                        
1127177    [cats are OK - purrr, apartment, laundry on si...
1127178    [townhouse, w/d in unit, no smoking, detached ...
1127179                                                   []
1127180    [cats are OK - purrr, dogs are OK - wooof, fur...
1127181        [apartment, laundry on site, detached garage]
Name: parsed_attrs, Length: 1127182, dtype: object

In [10]:
df.head()

,parsed_postingtitletext,parsed_map_address,added,PostUrl,parsed_price,parsed_housing,parsed_data_longitude,parsed_attrs,parsed_district,parsed_map_link,...,parsed_bedrooms,parsed_furnished,parsed_wd,parsed_type,parsed_laundryb,parsed_nthumbs,parsed_garagea,parsed_laundrys,parsed_garaged,parsed_catsok
0,$3150 / 2br - 1300ft2 - Sunset Oaks Townhome- ...,1249 Crescent Terrace,1538706586266,https://sfbay.craigslist.org/sby/apa/d/sunset-...,3150,2br - 1300ft2,-122.026428,"[townhouse, w/d in unit, no smoking, detached ...",sunnyvale,https://maps.google.com/?q=loc%3A+%31%32%34%39...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,$1769 / 2br - 875ft2 - ❄ Pittsburg Apartment L...,2205 E Leland Rd,1539451812116,https://sfbay.craigslist.org/eby/apa/d/pittsbu...,1769,2br - 875ft2,-121.864673,[apartment],"Pittsburg, CA",https://maps.google.com/?q=loc%3A+%32%32%30%35...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,$3000 / 2br - 1800ft2 - 2 Bedroom 1 Bath 1 Car...,375 Pope Street,1538947757137,https://sfbay.craigslist.org/sfc/apa/d/2-bedro...,3000,2br - 1800ft2,-122.439706,"[house, laundry in bldg, no smoking, attached ...",excelsior / outer mission,https://maps.google.com/?q=loc%3A+%33%37%35+Po...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,$2150 / 3br - 1200ft2 - Updated 1200 SQ FT 3 B...,2436 Baltic Dr,1585287116209,https://sfbay.craigslist.org/eby/apa/d/fairfie...,2150,3br - 1200ft2,-122.019156,"[house, w/d hookups, detached garage]",fairfield / vacaville,"https://www.google.com/maps/search/38.274790,-...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,$2150 / 1br - 726ft2 - FULLY RENOVATED 1BED 1B...,1182 E Street near 2nd,1587684611249,https://sfbay.craigslist.org/eby/apa/d/hayward...,2150,1br - 726ft2,-122.076737,"[cats are OK - purrr, dogs are OK - wooof, apa...",hayward / castro valley,"https://www.google.com/maps/search/37.671644,-...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df.shape

(1127182, 28)

In [12]:
def get_type(attrs):
    if not isinstance(attrs, list):
        return None
    types = set([
        "apartment", 
        "townhouse", 
        "loft", 
        "land", 
        "house", 
        "duplex", 
        "flat", 
        "condo", 
        "cottage/cabin"
    ])
    return ",".join(sorted(types & set(attrs)))

In [15]:
def prepare_data(df):
    new  = pd.DataFrame()
    new["PostUrl"] = df["PostUrl"]
    new["latitude"] = df["parsed_data_latitude"]
    new["longitude"] = df["parsed_data_longitude"]
    new["district"] = df["parsed_district"].map(lambda s: s.strip("()") if isinstance(s, str) else s)
    new["address"] = df["parsed_map_address"].map(lambda x: x if x != "(google map)" else None)
    new["housing"] = df["parsed_housing"]
    
    new["price"] = df["parsed_price"].map(lambda x: float(x) if x else x)
    
    # to numerical values
    new['price'] = pd.to_numeric(new['price'])
    new['longitude'] = pd.to_numeric(new['longitude'])
    new['latitude'] = pd.to_numeric(new['latitude'])
    
    new['bedrooms'] = new['housing'].str.extract('(\d+)br.*', expand=True)
    new['bedrooms'] = pd.to_numeric(new['bedrooms'])
    
    new['area'] = new['housing'].str.extract('(\d+)ft2', expand=True)
    new['area'] = pd.to_numeric(new['area'])
    
    new['type'] = df["parsed_attrs"].map(get_type)
    new['catsok'] = df["parsed_attrs"].map(lambda x: "cats are OK - purrr" in x)
    new['dogsok'] = df["parsed_attrs"].map(lambda x: "dogs are OK - wooof" in x)
    new['garagea'] = df["parsed_attrs"].map(lambda x: "attached garage" in x)
    new['garaged'] = df["parsed_attrs"].map(lambda x: "detached garage" in x)
    new['furnished'] = df["parsed_attrs"].map(lambda x: "furnished" in x)
    
    new['laundryb'] = df["parsed_attrs"].map(lambda x: "laundry in bldg" in x)
    new['laundrys'] = df["parsed_attrs"].map(lambda x: "laundry on site" in x)
    
    new['wd'] = df["parsed_attrs"].map(lambda x: "w/d in unit" in x)
    new['nthumbs'] = df["parsed_thumbs"].apply(lambda x: len(x) if isinstance(x, list) else 0)

    return new

In [16]:
short = prepare_data(df)

In [17]:
short

,PostUrl,latitude,longitude,district,address,housing,price,bedrooms,area,type,catsok,dogsok,garagea,garaged,furnished,laundryb,laundrys,wd,nthumbs
0,https://sfbay.craigslist.org/sby/apa/d/sunset-...,37.353754,-122.026428,sunnyvale,1249 Crescent Terrace,2br - 1300ft2,3150.0,2.0,1300.0,townhouse,False,False,False,True,False,False,False,True,18
1,https://sfbay.craigslist.org/eby/apa/d/pittsbu...,38.008321,-121.864673,"Pittsburg, CA",2205 E Leland Rd,2br - 875ft2,1769.0,2.0,875.0,apartment,False,False,False,False,False,False,False,False,8
2,https://sfbay.craigslist.org/sfc/apa/d/2-bedro...,37.710526,-122.439706,excelsior / outer mission,375 Pope Street,2br - 1800ft2,3000.0,2.0,1800.0,house,False,False,True,False,False,True,False,False,5
3,https://sfbay.craigslist.org/eby/apa/d/fairfie...,38.274790,-122.019156,fairfield / vacaville,2436 Baltic Dr,3br - 1200ft2,2150.0,3.0,1200.0,house,False,False,False,True,False,False,False,False,10
4,https://sfbay.craigslist.org/eby/apa/d/hayward...,37.671644,-122.076737,hayward / castro valley,1182 E Street near 2nd,1br - 726ft2,2150.0,1.0,726.0,apartment,True,True,False,False,False,False,False,True,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127177,https://sfbay.craigslist.org/sby/apa/d/ideal-1...,37.337841,-121.978369,santa clara,2865 Homestead Rd,1br - 750ft2,2195.0,1.0,750.0,apartment,True,False,False,False,False,False,True,False,12
1127178,https://sfbay.craigslist.org/eby/apa/d/fremont...,37.552033,-121.996875,fremont / union city / newark,4095 EGGERS DR,2br - 1000ft2,2800.0,2.0,1000.0,townhouse,False,False,False,True,False,False,False,True,8
1127179,https://sfbay.craigslist.org/eby/apa/d/your-ne...,NaN,NaN,fairfield / vacaville,NaN,/ 2br - 918ft2 -,1965.0,2.0,918.0,,False,False,False,False,False,False,False,False,18
1127180,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,37.787295,-122.392719,Rincon Hill,"340 Fremont St, San Francisco, CA 94105, USA",1br - 708ft2,4890.0,1.0,708.0,apartment,True,True,False,False,True,False,False,True,22


In [18]:
short.shape

(1127182, 19)

In [19]:
short.dropna(subset=["price"], inplace=True)
short.shape

(1122244, 19)

In [20]:
short["price"].describe()

count    1.122244e+06
mean     2.673215e+14
std      2.831898e+17
min      0.000000e+00
25%      2.100000e+03
50%      2.670000e+03
75%      3.495000e+03
max      3.000000e+20
Name: price, dtype: float64

In [21]:
# change all prices more than 9k to to 9 k
short.loc[short['price'] > 10000, "price"] = 11111

In [22]:
short["price"].describe()

count    1.122244e+06
mean     2.957790e+03
std      1.403968e+03
min      0.000000e+00
25%      2.100000e+03
50%      2.670000e+03
75%      3.495000e+03
max      1.111100e+04
Name: price, dtype: float64

In [23]:
short.shape

(1122244, 19)

In [24]:
short.drop_duplicates(inplace=True)
short.shape

(755567, 19)

In [25]:
short['price_bucket'] = short['price'].apply(lambda x: f"{x//500:02.0f}x500")

In [26]:
short.head()

,PostUrl,latitude,longitude,district,address,housing,price,bedrooms,area,type,catsok,dogsok,garagea,garaged,furnished,laundryb,laundrys,wd,nthumbs,price_bucket
0,https://sfbay.craigslist.org/sby/apa/d/sunset-...,37.353754,-122.026428,sunnyvale,1249 Crescent Terrace,2br - 1300ft2,3150.0,2.0,1300.0,townhouse,False,False,False,True,False,False,False,True,18,06x500
1,https://sfbay.craigslist.org/eby/apa/d/pittsbu...,38.008321,-121.864673,"Pittsburg, CA",2205 E Leland Rd,2br - 875ft2,1769.0,2.0,875.0,apartment,False,False,False,False,False,False,False,False,8,03x500
2,https://sfbay.craigslist.org/sfc/apa/d/2-bedro...,37.710526,-122.439706,excelsior / outer mission,375 Pope Street,2br - 1800ft2,3000.0,2.0,1800.0,house,False,False,True,False,False,True,False,False,5,06x500
3,https://sfbay.craigslist.org/eby/apa/d/fairfie...,38.274790,-122.019156,fairfield / vacaville,2436 Baltic Dr,3br - 1200ft2,2150.0,3.0,1200.0,house,False,False,False,True,False,False,False,False,10,04x500
4,https://sfbay.craigslist.org/eby/apa/d/hayward...,37.671644,-122.076737,hayward / castro valley,1182 E Street near 2nd,1br - 726ft2,2150.0,1.0,726.0,apartment,True,True,False,False,False,False,False,True,8,04x500


In [27]:
short["price_bucket"].describe()

count     755567
unique        22
top       04x500
freq      159544
Name: price_bucket, dtype: object

In [28]:
export_path = Path("./apthuntdata")
export_path.mkdir(exist_ok=True)
short.to_csv(export_path/"data.csv", index=False) 

In [29]:
short["price"].describe()

count    755567.000000
mean       2960.784224
std        1374.017075
min           0.000000
25%        2100.000000
50%        2695.000000
75%        3495.000000
max       11111.000000
Name: price, dtype: float64

In [ ]:
sns.distplot(short.loc[:, "price"])

In [ ]:
sns.distplot(short.loc[short["price"] < 20000, "price"])

In [ ]:
short[short.price > 15000].describe()

In [ ]:
1094/578376

In [ ]:
from fastai2.tabular.all import *

In [ ]:
def mean_abs_diff(inp, target):
    return (inp - target).abs().mean()
def min_abs_diff(inp, target):
    return (inp - target).abs().min()
def max_abs_diff(inp, target):
    return (inp - target).abs().max()

In [ ]:
learn_inf = load_learner('cltab.pkl')

In [ ]:
short.iloc[0]

In [ ]:
tp = pd.read_csv("apthuntdata/data.csv").loc[0]

In [ ]:
del tp["PostUrl"]
del tp["address"]
del tp["price"]
del tp["price_bucket"]

In [ ]:
tp

In [ ]:
prediction = learn_inf.predict(tp)
i = 0
for p in prediction:
    print("-" * 11)
    print(i)
    i += 1
    print(p)
    print(type(p))

In [ ]:
pd.read_csv("apthuntdata/data.csv").loc[0]

In [ ]:
topred = pd.read_csv("apthuntdata/data.csv")
del topred["PostUrl"]
del topred["address"]
del topred["price"]
del topred["price_bucket"]

In [ ]:
learn_inf.predict(topred)

In [ ]:
topred.loc[0:1]

In [ ]:
topred.apply(lambda x: f"{x['latitude']} + {x['district']}", axis=1)

In [ ]:
topred["price"] = topred.apply(learn_inf.predict, axis=1)